In [ ]:
import hashlib
import itertools
import math
import random

from collections import Counter
from functools import partial
from pathlib import Path
from sys import getsizeof
from time import time, process_time_ns

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline
mpl.rcParams['figure.dpi']     = 100
mpl.rcParams['figure.figsize'] = [10, 5]

px.defaults.height = 600

if not hasattr(math, 'comb'):
    def math_comb(n, r):
        f = math.factorial
        return f(n) // (f(r) * f(n - r))


    math.comb = math_comb

import fpfz
import primes
from itertools_recipes import take, first_true

In [ ]:
pg     = primes.Generator()
pp     = primes.Power(pg)
calc_u = fpfz.UniverseSizeCalculator(pg)
calc_m = fpfz.MemoryCalculator(pg, calc_u)
matgen = fpfz.MatrixGenerator(calc_u, calc_m, pg, rotate=False)

rng = np.random.default_rng()

In [ ]:
dicts_dir = Path('.')
dict_format = 'calc_m_{}.pkl'

def load_dict(d=None):
    if not d:
        d = {}
    for l in dicts_dir.glob(dict_format.format('*')):
        d.update(fpfz.read_from_disk(l))
        print(f'loaded {l}, |d| = {len(d)}')
    print('done')
    return d

In [ ]:
calc_m.rec_cache.update(load_dict())

In [ ]:
def dump_dict():
    print('dumping dict')
    fpfz.write_to_disk(
        dicts_dir / dict_format.format(int(time()*100)),
        calc_m.rec_cache
    )

# Runtime comparisons

#### Load csvs

In [ ]:
workdir = Path('space_measure/')

In [ ]:
def prepare_single_rowed_csv(df):
    df = df.melt(id_vars=['n', 'alg', 'm'], var_name='method', value_name='bits')
    return df

df = pd.concat(
    map(prepare_single_rowed_csv, map(pd.read_csv, workdir.glob('*.csv')))
, ignore_index=True).sort_values('n')

df

In [ ]:
px.line(df, x='n', color='alg', y='bits', log_y=True, log_x=True, facet_col='method')

In [ ]:
df[(df.alg=='REC7k14') & (df.n==2**20)]

In [ ]:
value = 'bits'
for method in df.method.unique():
    csv_df = df[df.method==method].copy()
    csv_df = csv_df.pivot_table(index='n', columns=['alg'], values=[value])[value]

    name = (
        'Matrix' if method=='M' else
        'Sparse' if method=='S' else
        'Column' if method=='C' else
        '??????'
    )
    csv_df.to_csv(f'space_measure_{name}.csv')

In [ ]:
workdir = Path('measuring_clean/')

In [ ]:
def prepare_single_rowed_csv(df):
    df = df.melt(id_vars=['n', 'rep'], var_name='alg', value_name='total_time_ns')
    return df

df = pd.concat(
    map(prepare_single_rowed_csv, map(pd.read_csv, workdir.glob('*.csv')))
, ignore_index=True)#.set_index('n').sort_index()

def gen_mmh_df(df):
    mmh_df = df[df.alg == df.iloc[df.n.argmax()].alg].copy()
    mmh_df.alg = 'C_MMH3'
    mmh_df.rep = 1
    mmh_df.total_time_ns = 2717.824865 * 10**6
    return mmh_df

df = pd.concat([df, gen_mmh_df(df)])

In [ ]:
df = df.pivot_table(index=['n', 'alg'], values='total_time_ns')['total_time_ns'].reset_index()

df['total_time_us'] = df['total_time_ns'] / 10**3
df['total_time_ms'] = df['total_time_ns'] / 10**6

df['mean_time_us'] = df['total_time_us'] / 10**6
df['mean_time_ms'] = df['total_time_ms'] / 10**6

df

In [ ]:
px.line(df, x='n', color='alg', y='mean_time_us', log_y=True, log_x=True)

In [ ]:
value = 'mean_time_us'
csv_df = df.copy()
csv_df.alg = csv_df.alg.str[2:]
csv_df = csv_df.pivot_table(index='n', columns='alg', values=[value])[value]
csv_df.to_csv(f'time_measure_clean.csv')

In [ ]:
workdir = Path('measuring/')

In [ ]:
def prepare_single_rowed_csv(df):
    df = df.melt(id_vars=['n', 'm', 'rep'], var_name='alg', value_name='total_time_ns')
    df['tag'] = df.alg[~df.alg.str.startswith('IBLT')][0][2:-3]
    return df

df = pd.concat(
    map(prepare_single_rowed_csv, map(pd.read_csv, workdir.glob('*.csv')))
, ignore_index=True)#.set_index('n').sort_index()
df

In [ ]:
df = df.pivot_table(index=['n', 'alg', 'm', 'tag'], values='total_time_ns', aggfunc='mean')['total_time_ns'].reset_index()
# df = df.groupby(['n', 'alg', 'm']).mean()['total_time_ns'].reset_index()
iblt_mask = df.alg.str.startswith('IBLT')
df.loc[iblt_mask, 'alg'] = 'C_' + df.loc[iblt_mask, 'alg']

df['method'] = df.alg.str[:1]
df['access'] = df.alg.str[-2:-1]
df['alg']    = df.alg.str[2:-3]
df['n_str']  = df.n.astype(str)

df['total_time_us'] = df['total_time_ns'] / 10**3
df['total_time_ms'] = df['total_time_ns'] / 10**6

df['mean_time_us'] = df['total_time_us'] / 10**6
df['mean_time_ms'] = df['total_time_ms'] / 10**6

# df['tag'] = df['alg']

# our_algs = [alg for alg in df.alg.unique() if not alg.startswith('IBLT')]

# for alg in our_algs:
#     df.loc[df.m.isin(df[df['alg']==alg].m.unique()), 'tag'] = alg

df

#### Compare array vs. list vs. iterable

In [ ]:
px.bar(
    df[df.n.isin((2**8, 2**10, 2**15, 2**20, 2**28))],
    x='method',
    y='total_time_ns',
    color='access',
    barmode='group',
    # pattern_shape='tag',
    facet_col='alg',
    facet_row='n_str',
)

In [ ]:
pt = df[~df.alg.str.startswith('IBLT')].pivot_table(index=['n', 'alg', 'm', 'method'], values=['total_time_ns'], columns=['access'])['total_time_ns']

In [ ]:
print('maximum deviation from best access type:')
print(f'{1 - (pt.min(1)/pt.max(1)).min():%}')

In [ ]:
pt.idxmin(1).value_counts()

#### Iterable leads the charts, we'll use it

In [ ]:
df = df[df['access']=='i'].drop(columns='access')

### Plots

In [ ]:
px.line(df, x='m', color='method', y='total_time_ms', facet_col='alg', facet_row='tag', log_y=True, log_x=True)

In [ ]:
px.line(df, x='n', color='method', y='total_time_ms', facet_col='alg', facet_row='tag', log_y=True, log_x=True)

In [ ]:
dfc = df[(df.tag=='d7') & (df.n==256)][['alg', 'method', 'mean_time_ms']]
dfc['total_of_1024'] = dfc['mean_time_ms']*1024
dfc

In [ ]:
1.134607/1.313365

In [ ]:
px.line(df, x='m', y='n', color='tag', log_y=True, log_x=True)

In [ ]:
value = 'mean_time_us'
for tag in df.tag.unique():
    csv_df = df[df.tag==tag].copy()
    csv_df.alg += '[' + csv_df.method + ']'
    csv_df = csv_df.pivot_table(index='m', values=['n']).join(
        csv_df.pivot_table(index='m', columns=['alg'], values=[value])[value]
    )

    csv_df.rename(columns=lambda x: (
        x[:4] if x.startswith('IBLT')
        else x if not (exp:=fmt.match(x))
        else 'Matrix' if exp[1]=='M'
        else 'Sparse' if exp[1]=='S'
        else 'Column' if exp[1]=='C'
        else x
    ), inplace=True)
    csv_df.to_csv(f'time_measure_{tag}.csv')

## Explorations

In [ ]:
def mapping_matrix_to_set_of_tuples(M):
    return {tuple(v) for v in M}

def verify_column_generator_d3(m):
    n = calc_u.d3_construction(m)
    d3 = matgen.recursive_d3(m)

    calc_c_d3 = fpfz.ColumnGeneratorD3(n=n, m=m)
    d3_a = np.zeros_like(d3)
    d3_i = np.zeros_like(d3)
    d3_g = np.zeros_like(d3)

    for i in range(n):
        calc_c_d3.array(i, d3_a[i])
        d3_i[i][calc_c_d3.list(i)] = 1
        d3_g[i][list(calc_c_d3.iterable(i))] = 1

    set_of_tuples_m = mapping_matrix_to_set_of_tuples(d3)
    for i,mat in enumerate((d3_a, d3_i, d3_g)):
        assert set_of_tuples_m == mapping_matrix_to_set_of_tuples(mat), i

def verify_column_generator_k(m, d, k):
    print(locals())
    n = calc_u.recursive(m, d, calc_m, k)
    M = matgen.recursive(n, d, k)

    calc_c = fpfz.ColumnGeneratorK(n, d, k, mc=calc_m)
    M_a = np.zeros_like(M)
    M_i = np.zeros_like(M)
    M_g = np.zeros_like(M)

    for i in range(n):
        calc_c.array(i, M_a[i])
        # M_i[i][calc_c.list(i)] = 1
        # M_g[i][list(calc_c.iterable(i))] = 1

    set_of_tuples_m = mapping_matrix_to_set_of_tuples(M)
    # for i,mat in enumerate((M_a, M_i, M_g)):
    for i,mat in enumerate((M_a,)):
        assert set_of_tuples_m == mapping_matrix_to_set_of_tuples(mat), f'\n\n{M}\n\n{M_a}'

def verify_column_generator_hamming(m):
    n = calc_u.ex_hamming(m)

    calc_c = fpfz.ColumnGeneratorHamming(m=m)
    h_a = np.zeros((n, m), dtype=int)
    h_i = np.zeros((n, m), dtype=int)
    h_g = np.zeros((n, m), dtype=int)

    for i in range(n):
        calc_c.array(i, h_a[i])
        h_i[i][calc_c.list(i)] = 1
        h_g[i][list(calc_c.iterable(i))] = 1

    set_of_tuples_m = mapping_matrix_to_set_of_tuples(h_a)
    for i,mat in enumerate((h_i, h_g)):
        assert set_of_tuples_m == mapping_matrix_to_set_of_tuples(mat), i

for m in list(range(3, 12)) + [17, 20, 27]:
    verify_column_generator_d3(m)
    verify_column_generator_hamming(m)

# verify_column_generator_k(20, 5, 5)

print('SUCCESS')

In [ ]:
workdir = Path('measuring')
workdir.mkdir(parents=True, exist_ok=True)

In [ ]:
def gen(m, reps=2, sample_size=10**4):
    print(f'generating for {m =:3}')
    n = calc_u.d3_construction(m_bits=m)

    n = min(n, 2**32)

    d3_mappers = {
        # 'IBLT': fpfz.HashMapper(m),
        # 'd3M': fpfz.MatrixIndicesMapper(matgen.recursive_d3(m=m)),
        # 'd3C': fpfz.ColumnGeneratorD3(m=m, n=n),
        'HMC'  : fpfz.ColumnGeneratorHamming(m=m),
        'd3k5' : fpfz.ColumnGeneratorK(n=calc_u.recursive(
            m=m, d=3, k=5, mem_calc=calc_m), d=3, k=5 , mc=calc_m
        ),
        'd5k7' : fpfz.ColumnGeneratorK(n=calc_u.recursive(
            m=m, d=5, k=7, mem_calc=calc_m), d=5, k=7 , mc=calc_m
        ),
        'd7k15': fpfz.ColumnGeneratorK(n=calc_u.recursive(
            m=m, d=7, k=15, mem_calc=calc_m), d=7, k=15 , mc=calc_m
        ),
    }
    results = Counter({'m': m, 'reps': reps})
    for rep in range(reps):
        print(f'{rep=}')
        for name, mapper in d3_mappers.items():
            # results[name + '[l]'] += fpfz.measure_insertions(m, mapper, universe_generator(n, sample_size), process_time_ns, fpfz.insertion_from_list )
            results[name + '[a]'] += fpfz.measure_insertions(m, mapper, universe_generator(n, sample_size), process_time_ns, fpfz.insertion_from_array)

    pd.DataFrame([results]).to_csv(workdir / f'{m}_{process_time_ns()}.csv', index=False)
    return results

In [ ]:
d3results = []

In [ ]:
for m in [
    3, 5, 10, 15, 20, 30, 40, 50, 100, 200, 500, 1000, 1500,
]:
    d3results.append(gen(m, reps=10, sample_size=10**6))

In [ ]:
d3results = pd.DataFrame.from_records(d3results).set_index('m')#.drop(columns='IBLT[a]')
d3results = d3results.divide(d3results['IBLT[l]'], axis="index")
d3results

In [ ]:
d3results.plot();

In [ ]:
# pd.concat([pd.read_csv(csv) for csv in workdir.glob('*.csv')]).set_index('m').to_csv('measuring.csv')

In [ ]:
d3results = pd.read_csv('measuring.csv').set_index('m').sort_index()
d3results = d3results.divide(d3results['reps'], axis='index').drop(columns=['reps'])
d3results = d3results.divide(d3results['IBLT[l]'], axis="index")
d3results

In [ ]:
d3results.drop(columns=['d3C[l]']).plot(logx=True, logy=True);

In [ ]:
d3results.drop(columns=['d3C[l]']).to_csv('time_measure.csv')

### Full vs. sparse matrix

In [ ]:
def size(a):
    return a.nbytes

def size_MB(a):
    return size(a)/2**20

In [ ]:
n = 2**16
k = 5
d = 7
M = matgen.recursive(n=n, d=d, k=k)
size_MB(M)

In [ ]:
m = M.shape[1]
m

In [ ]:
w = k
if w is None:
    w = M.sum(1).max()

lM = np.empty((n, w), dtype=np.uint16)
for i,e in enumerate(M):
    lM[i] = e.nonzero()[0]

In [ ]:
lM

In [ ]:
del M

In [ ]:
size_MB(lM)

In [ ]:
n*k*2/2**20

In [ ]:
counts  = np.zeros(m, dtype=int)
xorsums = np.zeros(m, dtype=int)
ysums   = np.zeros(m, dtype=int)

In [ ]:
%%timeit
outs = [b'42hf']
for i in range(4):
    outs.append(hashlib.sha1(outs[-1]).digest()[:4])

for out in outs:
    counts [int.from_bytes(out, 'big') % m] += 1
    xorsums[int.from_bytes(out, 'big') % m] ^= int.from_bytes(out, 'big')
    ysums  [int.from_bytes(out, 'big') % m] ^= int.from_bytes(out, 'big')

In [ ]:
%%timeit
outs = lM[54]

for out in outs:
    counts [out % m] += 1
    xorsums[out % m] ^= out
    ysums  [out % m] ^= out

In [ ]:
reps = 10

In [ ]:
start = time()
for _ in range(reps):
    for i in range(n):
        outs = [f'{i}'.encode()]
        for i in range(k):
            outs.append(hashlib.sha1(outs[-1]).digest()[:k])

        for out in outs:
            outint = int.from_bytes(out, 'big')
            counts [outint % m] += 1
            xorsums[outint % m] ^= outint
            ysums  [outint % m] ^= outint

total = time() - start
total

In [ ]:
start = time()
for _ in range(reps):
    for i in range(n):
        outs = lM[i]

        for out in outs:
            counts [out % m] += 1
            xorsums[out % m] ^= out
            ysums  [out % m] ^= out
total = time() - start
total

# Comparisons

## IBLT vs DFFZ

In [ ]:
def rand_elements(n, N):
    return rng.choice(n, N, replace=False)

def simulations(n, Ns, tests, r):
    a = np.empty((len(tests), len(Ns), r), dtype=bool)

    for iN,N in enumerate(Ns):
        for j in range(r):
            e = rand_elements(n, N)
            for it, t in enumerate(tests):
                a[it, iN, j] = t(e)
    return a

def gen_tests(name2mat: dict):
    tests = {k: fpfz.MatrixDecoder(v).is_decodable for k,v in name2mat.items()}
    return tests, pd.DataFrame({name:mat.shape for name, mat in name2mat.items()}, index=('n','m')).transpose()

def gen_tests_iblt(mks):
    def test(s, m, k):
        n = len(s)
        return fpfz.MatrixDecoder(matgen.iblt(m=m, k=k, n=n)).is_decodable(np.arange(n))
    return {f'{m=}_{k=}_RND_iblt': partial(test, m=m, k=k) for m,k in mks}

def plot(results, Ns, tests, d, ax=None):
    df = pd.DataFrame(results.T, index=pd.Index(Ns, name='N'), columns=tests)
    ax = df.plot(ylabel='success probability', title=f'{d=}', ax=ax)
    ax.plot([d,d],[df.values.min(),1])
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    return df


In [ ]:
ks = [3]
reps = 1000
Ns = np.arange(2, 30)

In [ ]:
d = 3
m = 15
n = 25

matrices = {
    f'{m=}_{d=}_k=3_rec'  : matgen.recursive(n=n, d=d, k=3),
    # f'{m=}_{d=}_k=5_rec': matgen.recursive(n=n, d=d, k=5),
    f'{m=}_{d=}_ols'      : matgen.ols(s=5, d=2),
} | {
    f'{m=}_{k=}_MM3_iblt' : matgen.iblt(m=m, n=n, k=k)
    for k in ks
}
tests, df = gen_tests(matrices)
df

In [ ]:
results = simulations(n, Ns, tests.values(), reps).mean(2)
df=plot(results, Ns, tests.keys(), d)
# df.to_csv(f'dffz_vs_iblt_d{d}_m{m}_n{n}.csv')

In [ ]:
d=5
n=512
m=71
# mm=130
k=4

matrices = {
    f'{m=}_{d=}_rec'     : matgen.recursive(n=n, d=d),
    # f'm={mm}_d={d+2}_rec'  : matgen.recursive(n=n, d=d+2),
} | {
    # f'{m=}_{k=}_MM3_iblt' : matgen.iblt(m=m, n=n, k=k),
    f'{m=}_k={k-1}_MM3_iblt' : matgen.iblt(m=m, n=n, k=k-1),
}
tests, df = gen_tests(matrices)
df

In [ ]:
results = simulations(n, Ns, tests.values(), reps*13).mean(2)
df=plot(results, Ns, tests.keys(), d)
df.to_csv(f'dffz_vs_iblt_d{d}_m{m}_n{n}.csv')

In [ ]:
d=5
m=64
n=calc_u.recursive(m, 5, calc_m)
k=4

matrices = {
    f'{m=}_d=3_{k=}_rec'  : matgen.recursive(n=490, d=3, k=k),
    f'{m=}_{d=}_rec'      : matgen.recursive(n=n, d=d),
} | {
    f'{m=}_{k=}_MM3_iblt' : matgen.iblt(m=m, n=n, k=k)
}
tests, df = gen_tests(matrices)
df

In [ ]:
results = simulations(n, Ns, tests.values(), reps).mean(2)
df=plot(results, Ns, tests.keys(), d)
df.to_csv(f'dffz_vs_iblt_d{d}_m{m}_n{n}.csv')

In [ ]:
d=5
m=64
n=calc_u.recursive(m, 5, calc_m)
k=4

matrices = {
    f'{m=}_{d=}_rec'      : matgen.recursive(n=n, d=d),
} | {
    f'{m=}_{k=}_MM3_iblt' : matgen.iblt(m=m, n=n, k=k)
}
tests, df = gen_tests(matrices)
df

In [ ]:
Ns = np.arange(1,25)

In [ ]:
results = simulations(n, Ns, tests.values(), 3000).mean(2)
df=plot(results, Ns, tests.keys(), d)
df.to_csv(f'dffz_vs_iblt_d{d}_m{m}_n{n}.csv')

## Large universe

In [ ]:
def rand_hash_collision(to_n, n):
    return len(np.unique(rng.choice(to_n, size=n, replace=True))) < n

def hash_collision_prob_emp(to_n, n, reps=2000):
    return sum(itertools.starmap(rand_hash_collision, itertools.repeat((to_n, n), reps)))/reps


In [ ]:
u = 2**8
N = 5

# x = np.logspace(1, 32, dtype=int, base=2)
x = np.arange(32, dtype=int)
plt.plot(x, [fpfz.hash_collision_prob(i, u) for i in x], label='p')
plt.plot(x, [hash_collision_prob_emp(u, N) for N in x], label='e1')
plt.legend()

In [ ]:
x = np.arange(1, 10, dtype=int)

for exp in np.linspace(4, 32, num=5, dtype=int):
    u = 2**exp
    plt.plot(x, [1-hash_collision_prob_emp(u, i) for i in x], label=f'$2^{exp}$')
plt.legend()
# plt.yscale('log')

In [ ]:
def success_prob_with_collisions(m, N, reps=3000):
    n = calc_u.recursive(m=m, d=N, mem_calc=calc_m)
    # n = min(2**31 - 1, n)
    return 1 - hash_collision_prob_emp(n, N, reps=reps)

In [ ]:
Ns = np.arange(2, 6, dtype=int)

for exp in np.linspace(3, 5, num=3, dtype=int):
    m = 2**exp
    plt.plot(Ns, [success_prob_with_collisions(m, N) for N in Ns], label=f'$m=2^{exp}$')
plt.legend()
# plt.yscale('log')

In [ ]:
m=32
n=2**22
k=4

matrices = {
    # f'{m=}_d=3_{k=}_rec'  : matgen.recursive(n=490, d=3, k=k),
    # f'{m=}_{d=}_rec'      : matgen.recursive(n=n, d=d),
} | {
    f'{m=}_{k=}_MM3_iblt' : matgen.iblt(m=m, n=n, k=k)
}
tests, df = gen_tests(matrices)
df

In [ ]:
Ns = np.arange(2, 5)
reps=3000

**MEH**

In [ ]:
ax = plt.subplot(111)
ax.plot(Ns, [success_prob_with_collisions(m, N, reps=reps) for N in Ns], label=f'U{m=}')
ax.legend()

results = simulations(n, Ns, tests.values(), reps).mean(2)
df = plot(results, Ns, tests.keys(), 2, ax)

## $m$-$n$-$d$(-$k$) tradeoff

In [ ]:
def get_k_dict():
    return fpfz.read_from_disk(dicts_dir / 'recursive_k_dict.pkl')

rk_df = pd.DataFrame.from_records(
    (k + v for k,v in get_k_dict().items()),
    columns=('n', 'd', 'k', 'm', 'i', 'j')
)

In [ ]:
steiner_df = fpfz.Steiner.generate_df(pg, 10000)

In [ ]:
outdir = Path('csv')
outdir.mkdir(exist_ok=True)

def config_plots():
    plt.loglog()
    plt.legend()
    # plt.xlim([n[0], n[-1]])
    plt.title(f'd={d}')
    plt.ylabel('m (memory)')
    plt.xlabel('n (universe size)')

samples = 200

In [ ]:
def lower_61(n, d):
    return math.log2(
        sum(math.comb(n, i) for i in range((d+1)//2))
    )

In [ ]:
# d=3, no k

n = np.unique(np.logspace(.5, 8, samples, dtype=int))
m = np.unique(np.logspace(.5, 2, samples, dtype=int))
d=3

d3n = list(itertools.takewhile(lambda us: us<= n[-1], map(calc_u.d3_construction, m)))

xys = {
    'rec'       : (d3n   , m[:len(d3n)]                                 ),
    'ex_Hamming': (n     , list(map(calc_m.ex_hamming, n))              ),
    'EGH'       : (n     , [calc_m.egh(i, d-1) for i in n]              ),
    'L61'       : (n     , [lower_61(i, d) for i in n]              ),
}

for name, nms in xys.items():
    df = pd.DataFrame(nms).T.rename(columns={0:'n',1:'m'})
    plt.plot(*df.values.T, label=name)
    df.to_csv(f'csv/nm_{name}_d{d}.csv', index=False)

config_plots()

In [ ]:
# k

n = np.unique(np.logspace(1, 5, samples, dtype=int))
m = np.unique(np.logspace(1, 3.5, samples, dtype=int))
d=3

aldpcn = list(itertools.takewhile(lambda us: us<= n[-1], map(lambda i: calc_u.array_ldpc(i, s=d+1), m)))

def gen_rec_k(d, k):
    return rk_df[(rk_df.k==k) & (rk_df.d==d)].set_index('n')['m'].loc.__getitem__

rec_k3 = gen_rec_k(d, 3)
rec_k5 = gen_rec_k(d, 5)

xys = {
    'LDPC_arr'  : (aldpcn, m[:len(aldpcn)]                              ),
    # 'rec_k3_'   : (n     , [calc_m.recursive(i, d, k=3) for i in n]     ),
    'rec_k3'    : (n     , [rec_k3(i) for i in n]     ),
    'rec_k5'    : (n     , [rec_k5(i) for i in n]     ),
    'OLS'       : (n     , [calc_m.ols(i, d-1) for i in n]              ),
    'STNR'      : (n     , [calc_m.steiner(i, d, steiner_df) for i in n]),
    'L61'       : (n     , [lower_61(i, d) for i in n]              ),
}

for name, nms in xys.items():
    df = pd.DataFrame(nms).T.rename(columns={0:'n',1:'m'})
    plt.plot(*df.values.T, label=name)
    df.to_csv(f'csv/nm_{name}_d{d}.csv', index=False)

config_plots()

In [ ]:
# no k
d=15
n = np.unique(np.logspace(.5, 6.2, samples, dtype=int))

xys = {
    # 'LDPC_arr': (aldpcn, m[:len(aldpcn)]                              ),
    'rec'     : (n     , [calc_m.recursive(i, d) for i in n]          ),
    'EGH'     : (n     , [calc_m.egh(i, d-1) for i in n]              ),
    # 'OLS'     : (n     , [calc_m.ols(i, d-1) for i in n]              ),
    # 'STNR'    : (n     , [calc_m.steiner(i, d, steiner_df) for i in n]),
    'L61'       : (n     , [lower_61(i, d) for i in n]              ),
}

for name, nms in xys.items():
    df = pd.DataFrame(nms).T.rename(columns={0:'n',1:'m'})
    plt.plot(*df.values.T, label=name)
    df.to_csv(f'csv/nm_{name}_d{d}.csv', index=False)

config_plots()

In [ ]:
# k
d=7

n = np.unique(np.logspace(1, 5, samples, dtype=int))
m = np.unique(np.logspace(1, 4, samples, dtype=int))

aldpcn = list(itertools.takewhile(lambda us: us<= n[-1], map(lambda i: calc_u.array_ldpc(i, s=d+1), m)))

rec_k3 = gen_rec_k(d, 3)
rec_k5 = gen_rec_k(d, 5)

xys = {
    'LDPC_arr': (aldpcn, m[:len(aldpcn)]                              ),
    # 'rec_k3_'   : (n     , [calc_m.recursive(i, d, k=3) for i in n]     ),
    'rec_k5'   : (n     , [calc_m.recursive(i, d, k=5) for i in n]     ),
    'rec_k7'   : (n     , [calc_m.recursive(i, d, k=7) for i in n]     ),
    # 'rec_k3'    : (n     , [rec_k3(i) for i in n]     ),
    # 'rec_k5'    : (n     , [rec_k5(i) for i in n]     ),
    # 'EGH'     : (n     , [calc_m.egh(i, d-1) for i in n]              ),
    'OLS'     : (n     , [calc_m.ols(i, d-1) for i in n]              ),
    'STNR'    : (n     , [calc_m.steiner(i, d, steiner_df) for i in n]),
    'L61'       : (n     , [lower_61(i, d) for i in n]              ),
}

for name, nms in xys.items():
    df = pd.DataFrame(nms).T.rename(columns={0:'n',1:'m'})
    plt.plot(*df.values.T, label=name)
    df.to_csv(f'csv/nm_{name}_d{d}.csv', index=False)

config_plots()

In [ ]:
# k
d=15

n = np.unique(np.logspace(1, 5, samples, dtype=int))
m = np.unique(np.logspace(1, 4, samples, dtype=int))

# aldpcn = list(itertools.takewhile(lambda us: us<= n[-1], map(lambda i: calc_u.array_ldpc(i, s=d+1), m)))

rec_k3 = gen_rec_k(d, 3)
rec_k5 = gen_rec_k(d, 5)

xys = {
    # 'LDPC_arr': (aldpcn, m[:len(aldpcn)]                              ),
    # 'rec_k3_'   : (n     , [calc_m.recursive(i, d, k=3) for i in n]     ),
    'rec_k5'   : (n     , [calc_m.recursive(i, d, k=5) for i in n]     ),
    'rec_k7'   : (n     , [calc_m.recursive(i, d, k=7) for i in n]     ),
    # 'rec_k3'    : (n     , [rec_k3(i) for i in n]     ),
    # 'rec_k5'    : (n     , [rec_k5(i) for i in n]     ),
    # 'EGH'     : (n     , [calc_m.egh(i, d-1) for i in n]              ),
    'OLS'     : (n     , [calc_m.ols(i, d-1) for i in n]              ),
    'STNR'    : (n     , [calc_m.steiner(i, d, steiner_df) for i in n]),
    'L61'       : (n     , [lower_61(i, d) for i in n]              ),
}

for name, nms in xys.items():
    df = pd.DataFrame(nms).T.rename(columns={0:'n',1:'m'})
    plt.plot(*df.values.T, label=name)
    df.to_csv(f'csv/nm_{name}_d{d}.csv', index=False)

config_plots()

## $k$ values for non fixed $k$ constructions

In [ ]:
matgen.recursive_d3(7).sum(1).mean()

In [ ]:
def mean_weight(c):
    return np.array(list(c.items())).prod(1).sum()/sum(c.values())

mean_weight(matgen.recursive_d3_weights(1000))

In [ ]:
samples = 10
ms = np.unique(np.logspace(1, 3.5, samples, dtype=int))

plt.plot(ms, [max(matgen.recursive_d3_weights(m)) for m in ms])
plt.plot(ms, [mean_weight(matgen.recursive_d3_weights(m)) for m in ms])
plt.loglog()

# Bounds

* $n$ - size of the code (the memory)
* $M$ - size of the universe (log(M)=k)
* $d$ - every subset up to d (including) is not a stopping set

### M. Schwartz lower bound for stopping redundancy

In [ ]:
def w(n, i, d_co):
    frac = int(math.ceil((n+1)/i)-1)
    return max(frac, d_co)

In [ ]:
w(6, 1, 3)

In [ ]:
def lower_stopping_r(n = 30, d = 5, d_co = 5):
    l = []

    for i in range(1, d):
        c = math.comb
        wi = w(n, i, d_co)
        exp = c(n, i) / (wi * c(n-wi, i-1))
        l.append(math.ceil(exp))
    low_ro = max(l)
    return low_ro, l

lower_stopping_r()

### Bounds for ZFD

In [ ]:
zfd_methods = calc_M.ols, calc_M.egh

In [ ]:
for f in zfd_methods:
    print(f'{f.__name__:20} n>={math.ceil(f(7, 24))}')

In [ ]:
lower_stopping_r(12)

In [ ]:
# Etzion, simplex code PCM
v = np.arange(1,20)
print('       n      M      d')
print(np.array([
    2**v - v - 1,
    2**v - 1,
    2**(v - 1)
]).T)

### Bounds from MDS (Scwartz)
The $d$ here stands for minimal stopping set size

In [ ]:
# def lower_mds(n, d):
#     q = n//2
#     return math.ceil(math.comb(n//(q-1), d-2)/(d-1))*math.log2(q)

def lower_mds(n, d):
    return math.ceil(math.comb(n, d-2)/(d-1))

def upper_mds(n, d):
    return math.ceil(
        math.comb(n, d-2)
        *
        (max(d-1, n-d+2))/n
    )

In [ ]:
plt.figure()
plt.loglog()
# for d in range(3, 20, 4):
for d in [5]:
    lmds_d = partial(lower_mds, d=d)
    umds_d = partial(upper_mds, d=d)
    n = np.arange(d+2, 1000)
    l = np.fromiter(map(lmds_d, n), dtype=int)
    u = np.fromiter(map(umds_d, n), dtype=int)
    
    plt.plot(n, l, '^', label=f'l {d=}')
    plt.plot(n, [calc_M.egh(i, d) for i in n], label=f'EGH {d=}')
    plt.plot(n, [calc_M.ols(i, d) for i in n], label=f'OLS {d=}')
    plt.plot(n, u, 'v', label=f'u {d=}')

plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.yscale('log')
for n in range(4, 100, 20):
    umds = partial(upper_mds, n=n)
    d = np.arange(2, n - 2)
    u = np.array([umds(n=n, d=d_) for d_ in d])

    plt.plot(d, u, 'v', label=f'{n=}')

plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.yscale('log')

n=40
umds = partial(upper_mds, n=n)
lmds = partial(lower_mds, n=n)
d = np.arange(2, n - 2)
u = np.array([umds(n=n, d=d_) for d_ in d])
l = np.array([lmds(n=n, d=d_) for d_ in d])

plt.plot(d, u, '^', label='UPPER')
plt.plot(d, l, 'v', label='LOWER')

plt.legend()
plt.show()

# Constructions

In [ ]:
def I(n):
    return np.eye(n, dtype=int)
def C0(n):
    return np.zeros((n, 1), dtype=int)
def C1(n):
    return np.ones((n, 1), dtype=int)

## Manual exploration

In [ ]:
h = {}
def n(m):
    return h[m].shape[0]

# h[1] = I(1)
# h[2] = I(2)
h[3] = np.vstack((I(3), C1(3).T))
h[4] = np.block([
    [I(3), C0(3)],
    [h[3], C1(4)],
])
h[5] = np.block([
    [I(3), C0(3)   , C0(3)   ],
    [h[3], C1(n(3)), C0(n(3))],
    [h[3], C0(n(3)), C1(n(3))],
])
h[6] = np.block([
    [I(3), C0(3), C0(3)   , C0(3)   ],
    [h[4]       , C1(n(4)), C0(n(4))],
    [h[4]       , C0(n(4)), C1(n(4))],
])
h[7] = np.block([
    [I(3), C0(3), C0(3), C0(3), C0(3)],
    [h[5]       , C1(n(5)), C0(n(5))],
    [h[5]       , C0(n(5)), C1(n(5))],
])
h[8] = np.block([
    [I(3), C0(3), C0(3), C0(3), C0(3), C0(3)],
    [h[6]       , C1(n(6)), C0(n(6))],
    [h[6]       , C0(n(6)), C1(n(6))],
])
h[9] = np.block([
    [I(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3)],
    [h[6]       , C1(n(6)), C0(n(6)), C0(n(6))],
    [h[6]       , C0(n(6)), C1(n(6)), C0(n(6))],
    [h[6]       , C0(n(6)), C0(n(6)), C1(n(6))],
])
h[10] = np.block([
    [I(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3)],
    [h[7]       , C1(n(7)), C0(n(7)), C0(n(7))],
    [h[7]       , C0(n(7)), C1(n(7)), C0(n(7))],
    [h[7]       , C0(n(7)), C0(n(7)), C1(n(7))],
])
h[11] = np.block([
    [I(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3)],
    [h[8]       , C1(n(8)), C0(n(8)), C0(n(8))],
    [h[8]       , C0(n(8)), C1(n(8)), C0(n(8))],
    [h[8]       , C0(n(8)), C0(n(8)), C1(n(8))],
])
h[12] = np.block([
    [I(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3), C0(3)],
    [h[9]       , C1(n(9)), C0(n(9)), C0(n(9))],
    [h[9]       , C0(n(9)), C1(n(9)), C0(n(9))],
    [h[9]       , C0(n(9)), C0(n(9)), C1(n(9))],
])
for m,H in h.items():
    assert H.shape[0]==len(set(map(tuple, H.tolist()))), f'duplicates! {m}'
    assert H.shape[1]==m, f'{m}!={H.shape[1]}'
    assert H.any(1).all(), f'Zero row in H[{m}]'
print('done sanity checks, no duplicates, shapes are fine, no zero vectors')

In [ ]:
mdf = pd.DataFrame.from_records(
    map(lambda H:
        H.shape +
        (calc.d3_construction(H.shape[1]),
         fpfz.is_pd(H.astype(bool), 3),
         np.linalg.matrix_rank(H)
        ), h.values()), columns=['n','m', 'exp. n', 'd=3?', 'rank']
).set_index('m')
if mdf['d=3?'].all():
    print('*'*5, 'all good', '*'*5)
mdf

## Constructive $d=3$

In [ ]:
for i in range(3,13):
    print(i, fpfz.is_pd(fpfz.recursive_d3(i).astype(bool), 3))
print('done')

## Constructive $d=4$

In [ ]:
d=4

In [ ]:
h_chk = np.array([
#   | <- old d3 ->|
    [1, 0, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0, 0], #1
    [0, 0, 1, 0, 0, 0, 0, 1], #2
    [1, 0, 0, 1, 0, 1, 0, 1], #3
    [0, 1, 0, 1, 0, 0, 0, 0], #4
    [0, 0, 1, 1, 0, 0, 0, 0], #5
    [1, 1, 1, 1, 0, 0, 1, 0], #6
    [1, 0, 0, 0, 1, 0, 0, 1], #7
    [0, 1, 0, 0, 1, 0, 0, 0], #8
    [0, 0, 1, 0, 1, 0, 1, 0], #9
    [1, 1, 1, 0, 1, 1, 0, 0], #10
])
# daniella's:
h_chk = np.vstack([
    I(7),
    np.array([
        [1, 1, 0, 0, 0, 0, 0], #7
        [0, 0, 1, 1, 0, 0, 0], #8
        [0, 0, 0, 0, 1, 1, 0], #9
        [1, 1, 1, 1, 1, 1, 1], #10
    ])
])
# Avi's:
h_chk = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 0],
    [1, 1, 0, 0, 1, 0],
])
# Avi's:
h_chk = np.array([
    [0, 1, 0, 0, 0, 0,    1, 1, 1,  0, 0, 0,    1, 1, 1, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 0, 1, 0, 0, 0,    1, 1, 0,  0, 0, 0,    1, 1, 1, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 0,    1, 0, 1,  0, 0, 0,    1, 1, 0, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 0, 0, 1, 0, 0,    1, 0, 0,  0, 0, 0,    1, 1, 0, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 1,    0, 1, 1,  0, 0, 0,    1, 0, 1, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 1, 1, 0, 0, 1,    0, 1, 0,  0, 0, 0,    1, 0, 1, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 0, 0, 1, 1, 0,    0, 0, 1,  0, 0, 0,    1, 0, 0, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [1, 1, 0, 0, 1, 0,    0, 0, 0,  0, 0, 0,    1, 0, 0, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
                                                            
    [0, 1, 0, 0, 0, 0,    0, 0, 0,  1, 1, 1,    0, 1, 1, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 0, 1, 0, 0, 0,    0, 0, 0,  1, 1, 0,    0, 1, 1, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 0,    0, 0, 0,  1, 0, 1,    0, 1, 0, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 0, 0, 1, 0, 0,    0, 0, 0,  1, 0, 0,    0, 1, 0, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 1,    0, 0, 0,  0, 1, 1,    0, 0, 1, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 1, 1, 0, 0, 1,    0, 0, 0,  0, 1, 0,    0, 0, 1, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
    [0, 0, 0, 1, 1, 0,    0, 0, 0,  0, 0, 1,    0, 0, 0, 1,  0, 0, 0, 0,  0, 0, 0, 0,],
    [1, 1, 0, 0, 1, 0,    0, 0, 0,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
                                                   
    [0, 1, 0, 0, 0, 0,    1, 1, 1,  0, 0, 0,    0, 0, 0, 0,  1, 1, 1, 1,  0, 0, 0, 0,],
    [0, 0, 1, 0, 0, 0,    1, 1, 0,  0, 0, 0,    0, 0, 0, 0,  1, 1, 1, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 0,    1, 0, 1,  0, 0, 0,    0, 0, 0, 0,  1, 1, 0, 1,  0, 0, 0, 0,],
    [0, 0, 0, 1, 0, 0,    1, 0, 0,  0, 0, 0,    0, 0, 0, 0,  1, 1, 0, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 1,    0, 1, 1,  0, 0, 0,    0, 0, 0, 0,  1, 0, 1, 1,  0, 0, 0, 0,],
    [0, 1, 1, 0, 0, 1,    0, 1, 0,  0, 0, 0,    0, 0, 0, 0,  1, 0, 1, 0,  0, 0, 0, 0,],
    [0, 0, 0, 1, 1, 0,    0, 0, 1,  0, 0, 0,    0, 0, 0, 0,  1, 0, 0, 1,  0, 0, 0, 0,],
    [1, 1, 0, 0, 1, 0,    0, 0, 0,  0, 0, 0,    0, 0, 0, 0,  1, 0, 0, 0,  0, 0, 0, 0,],
                                                                
    [0, 1, 0, 0, 0, 0,    0, 0, 0,  1, 1, 1,    0, 0, 0, 0,  0, 1, 1, 1,  0, 0, 0, 0,],
    [0, 0, 1, 0, 0, 0,    0, 0, 0,  1, 1, 0,    0, 0, 0, 0,  0, 1, 1, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 0,    0, 0, 0,  1, 0, 1,    0, 0, 0, 0,  0, 1, 0, 1,  0, 0, 0, 0,],
    [0, 0, 0, 1, 0, 0,    0, 0, 0,  1, 0, 0,    0, 0, 0, 0,  0, 1, 0, 0,  0, 0, 0, 0,],
    [1, 0, 0, 0, 0, 1,    0, 0, 0,  0, 1, 1,    0, 0, 0, 0,  0, 0, 1, 1,  0, 0, 0, 0,],
    [0, 1, 1, 0, 0, 1,    0, 0, 0,  0, 1, 0,    0, 0, 0, 0,  0, 0, 1, 0,  0, 0, 0, 0,],
    [0, 0, 0, 1, 1, 0,    0, 0, 0,  0, 0, 1,    0, 0, 0, 0,  0, 0, 0, 1,  0, 0, 0, 0,],
    [1, 1, 0, 0, 1, 0,    0, 0, 0,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
                                                   
    [0, 1, 0, 0, 0, 0,    1, 1, 1,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 1, 1, 1,],
    [0, 0, 1, 0, 0, 0,    1, 1, 0,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 1, 1, 0,],
    [1, 0, 0, 0, 0, 0,    1, 0, 1,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 1, 0, 1,],
    [0, 0, 0, 1, 0, 0,    1, 0, 0,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 1, 0, 0,],
    [1, 0, 0, 0, 0, 1,    0, 1, 1,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 0, 1, 1,],
    [0, 1, 1, 0, 0, 1,    0, 1, 0,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 0, 1, 0,],
    [0, 0, 0, 1, 1, 0,    0, 0, 1,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 0, 0, 1,],
    [1, 1, 0, 0, 1, 0,    0, 0, 0,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  1, 0, 0, 0,],
                                                              
    [0, 1, 0, 0, 0, 0,    0, 0, 0,  1, 1, 1,    0, 0, 0, 0,  0, 0, 0, 0,  0, 1, 1, 1,],
    [0, 0, 1, 0, 0, 0,    0, 0, 0,  1, 1, 0,    0, 0, 0, 0,  0, 0, 0, 0,  0, 1, 1, 0,],
    [1, 0, 0, 0, 0, 0,    0, 0, 0,  1, 0, 1,    0, 0, 0, 0,  0, 0, 0, 0,  0, 1, 0, 1,],
    [0, 0, 0, 1, 0, 0,    0, 0, 0,  1, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  0, 1, 0, 0,],
    [1, 0, 0, 0, 0, 1,    0, 0, 0,  0, 1, 1,    0, 0, 0, 0,  0, 0, 0, 0,  0, 0, 1, 1,],
    [0, 1, 1, 0, 0, 1,    0, 0, 0,  0, 1, 0,    0, 0, 0, 0,  0, 0, 0, 0,  0, 0, 1, 0,],
    [0, 0, 0, 1, 1, 0,    0, 0, 0,  0, 0, 1,    0, 0, 0, 0,  0, 0, 0, 0,  0, 0, 0, 1,],
    [1, 1, 0, 0, 1, 0,    0, 0, 0,  0, 0, 0,    0, 0, 0, 0,  0, 0, 0, 0,  0, 0, 0, 0,],
])
len(h_chk)

In [ ]:
sss = set(fpfz.pd_stopping_sets_generator(h_chk.astype(bool), d))
print('total stopping sets:', len(sss))
sss

In [ ]:
Counter([item for sublist in sss for item in sublist]).most_common(11)

In [ ]:
for i,r in enumerate(h_chk):
    print(i,r)

In [ ]:
h_chk.take(fpfz.get_pd_stopping_set(h_chk.astype(bool), d), 0)

## General $d$

### Check valitity

In [ ]:
ns = range(4, 100)
ds = range(4, 6)
total = len(ns)*len(ds)
print(f'{total=}')

for i, (d, n) in enumerate(itertools.product(ds, ns)):
    if i%20==0:
        print(f'{i/total:.3%}')
    if d > n:
        continue
    assert fpfz.is_pd(matgen.recursive(n, d).astype(bool), d), f'{n=}, {d=}'

### Custom

In [ ]:
cache = {}
shmez = {}

def recursive(n, d):
    if res := cache.get((n, d)):
        return res[0]
    
    if n <= d:
        return n
    if n == 1 or d == 1:
        return 1
    if d == 2:
        return math.ceil(math.log2(n+1))
    if d == 3:
        return calc_M.d3_construction(n)
      
    def m(n, d, i):
        ni = math.ceil(n/i)
        major = recursive(ni, d)
        minor = recursive(ni, d//2)
        return major + i*minor
    
    ms = [(m(n, d, i), i) for i in range(2, n+1)]
    best = min(min(ms), (n, -1))
    
    cache[(n, d)] = best
    shmez[(n, d)] = ms
            
    return best[0]

In [ ]:
recursive(1000000, 64)

In [ ]:
df = pd.DataFrame(((n,d,m,i) for (n,d),(m,i) in cache.items()), columns=('n','d','m','i'))

In [ ]:
df

### Explore $i$ histogram

In [ ]:
df.plot.scatter('n', 'i')

In [ ]:
df[(df.d == 64) & (df.n == 1000000)]

In [ ]:
v = np.array(shmez[(1000000, 64)]).T

In [ ]:
v = v[:, v[1] < 100]

In [ ]:
plt.xlabel('i')
plt.ylabel('m')
plt.plot(v[1], v[0]);

### What i,j are chosen in k,d construction?

In [ ]:
class Tracer:
    def __init__(self, mc):
        self.mc = mc

    def recursive_k(self, n, d, k):
        trace = [self.mc.recursive_k_ex(n, d, k)]
        m, i, j = trace[0]
    
        d = min(d, n)
    
        if i == 1 or k == 1 or d <= 2:
            return trace
    
        ni = math.ceil(n / i)
        trace.extend(self.recursive_k(ni, d   , k-j))
        trace.extend(self.recursive_k(ni, d//2, j  ))
        
        return trace

tracer = Tracer(calc_m)

In [ ]:
tracer.recursive_k(n=n, d=5, k=6)

## [OLD] For $d=3$, what is the best $m$ for given $n$ ($k=0$)

In [ ]:
mdf = out[(out.code=='PD') & (out.d==3)].loc[:,['n', 'm']].groupby(['n']).max().copy()
mdf.rename(columns={'m': 'm_sim'}, inplace=True)
mdf

Conjecture:

$m(n,3)=m(n-1,3)+n-1 = 1+\frac{n(n-1)}{2}=O(n^2)$

In [ ]:
def m1(n, d=3):
    assert d==3 and n > 0
    return int(1 + n*(n-1)/2)

In [ ]:
mdf['m1'] = mdf.index.map(m1)
mdf

This is not true! I got: $m(6)\geq19$.
This is my new conjecture:

$m(n, 3) = 4m(n - 3)+3$

In [ ]:
def m2(n, d=3):
    assert d==3
    if n<=3:
        return m1(n)
    return int(3 + 4*m(n-3))

In [ ]:
mdf['m2'] = mdf.index.map(m2)
mdf

This is not true, but it is still exponential.